### Type-Dispatch Design: Post Object-Oriented Programming for Julia

http://www.stochasticlifestyle.com/type-dispatch-design-post-object-oriented-programming-julia/

Le multiple dispatch est un concept simple : Les fonctions peuvent avoir plusieurs définitions tant que chaque définition accepte une combinaison différente d'arguments.

C'est quand une fonction a un comportement différent en fonction du type de ses arguments.

Une fonction dans Julia est essentiellement juste un nom. Associé à ce nom, il y a un certain nombre de méthodes, une pour chaque combinaison possible d'arguments. Ainsi, quand vous appelez une fonction dans Julia, elle cherchera dans une table de méthodes. Ces méthodes sont triées en fonction de la spécificité des types d'arguments. Par conséquent, elle essaiera d'abord de faire correspondre la signature de l'appel avec les types les plus spécifiques possibles. S'il ne trouve pas cela, il commencera à chercher des types légèrement plus abstraits plus bas dans la liste.


In [1]:
function sort(x::Array{String, 1}) # x is required to be a 1 dimensional array of strings.  
end
function sort(x::Array{Int, 1})
end
function sort(x::Array{Bool, 1})
end

sort (generic function with 3 methods)

La possibilité d'avoir plusieurs méthodes de même nom agissant différemment en fonction des paramètres permet d'éviter d'avoir à faire des if qui traitenet différemment en fonction du type de l'argument.

Imaginons que l'on veuille créer une fonction f qui square l'inputet calcule sa valeur modulo 4 et que la rapidité importe beaucoup. On impose aussi que f doit retourner un integer mais son input peut être un string, float ou int, et on ne connait pas le type de x sauf à l'exécution.

En python on ferait :
```py
def f_py(x):
    if type(x) == string:
        x = float(x)
    if type(x) == float:
        x = ceil(x)
    return x**2 % 4
```

En Julia on peut faire :


In [2]:
f(x::Int64) = x^2 %4
f(x::Float64) = f(ceil(Int64,x))
f(x::String) = f(parse(Float64,x))  

f (generic function with 3 methods)

Autre exemple, pour faire une fonction qui compte le nombre de ligne dans un fichier peu importe l'entrée (un fichier, plusieurs fichiers, des ou un fichier avec un filtre sur les extensions) on peut aussi faire ça simplement en Julia grâce aux multiple dispatch. On peut ajouter plusieurs fonctionnalités très rapidement sans modifier le code initial :

In [3]:
# compte les lignes sur un fichier
function countlines(fname::String)
    fstream = open(fname)
    num_lines = length(readlines(fstream))
    close(fstream)
    return num_lines
end
# compte les lignes de plusieurs fichiers
function countlines(files::Array{String, 1})
    return sum(map(countlines, files)) # map applique la fonction countlines à tous les fichiers dans files
end
# compte les lignes de plusieurs fichiers qui ont une certaine extension
function countlines(files::Array{String, 1}, ext::String)
    filtered = filter(x->endswith(x, ext), files) # filter va enlever tous les files qui respectent pas la condition
    return countlines(filtered) 
end
# compte lesl ignes d'un fichier avec une certaine extension
function countlines(file::String, ext::String)
    return endswith(file, ext) ? countlines(file) : 0
end

countlines (generic function with 4 methods)

#### Exemple avec des températures 

In [4]:
#import Base: +, -, promote_rule, convert, show
import Base: +,- # pour permettre de redéfinir la fonction + sans "shadower" la méthode + initiale
import Base: promote_rule, convert # pour permettre d'ajouter ces nouvelles definitions aux définitions initiales de ces méthodes sans les écraser

Déjà on définit les types :

In [5]:
abstract type Temperature end

struct Celsius <: Temperature
    value::Float64
end
struct Kelvin <: Temperature
    value::Float64
end
struct Fahrenheit <: Temperature
    value::Float64
end

Ensuite, nous utilisons la fonction promote_rule() pour enregistrer le type vers lequel deux types doivent être promus lorsqu'ils sont impliqués dans une expression arithmétique. Par exemple si on fait 1+1.5, une promote_rule va permettre de changer le type d'une variable pour être capable de faire l'opération. Ici, 1 serait converti en float pour avoir 1.0 

In [6]:
promote_rule(::Type{Kelvin}, ::Type{Celsius}) = Kelvin
# Type{Kelvin} prend le Type du type Kelvin
promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius
promote_rule(::Type{Fahrenheit}, ::Type{Kelvin}) = Kelvin

promote_rule (generic function with 127 methods)

Définissons ensuite les moyens de conversion entre chaque température :

In [7]:
convert(::Type{Kelvin},  t::Celsius)     = Kelvin(t.value + 273.15)
convert(::Type{Kelvin},  t::Fahrenheit)  = Kelvin(Celsius(t))
convert(::Type{Celsius}, t::Kelvin)      = Celsius(t.value - 273.15)
convert(::Type{Celsius}, t::Fahrenheit)  = Celsius((t.value - 32)*5/9)
convert(::Type{Fahrenheit}, t::Celsius)  = Fahrenheit(t.value*9/5 + 32)
convert(::Type{Fahrenheit}, t::Kelvin)   = Fahrenheit(Celsius(t))

convert (generic function with 197 methods)

Redéfinissons également les constructeurs des différentes températures pour pouvoir accepter d'autre types de températures

In [12]:
Kelvin(c::Celsius) = convert(Kelvin,c)
Kelvin(f::Fahrenheit) = convert(Kelvin,f)
Celsius(k::Kelvin) = convert(Celsius,k)
Celsius(f::Fahrenheit) = convert(Celsius,f)

# on a besoin de ces constructeurs car dans convert on fait Fahrenheit(Celsius(t)) par exemple
Fahrenheit(c::Celsius) = convert(Fahrenheit,c)
Fahrenheit(k::Kelvin) = convert(Fahrenheit,k)

Fahrenheit

Redéfinition des opérations arithmétiques :

In [9]:
# pour les opérations de même type :

+(x::T, y::T) where {T<:Temperature} = T(x.value + y.value)
-(x::T, y::T) where {T<:Temperature}= T(x.value - y.value)


- (generic function with 215 methods)

Pour traiter la combinaison de températures de types différents, nous utilisons le système de promotion de Julia, qui s'appuie sur convert() et promote_rule(). La bibliothèque standard de Julia a une fonction promote(x,y) qui rendra x et y dans le même type selon les règles de promotion. Ainsi nous définissons l'addition et la soustraction entre deux températures abstraites :

In [10]:
+(x::Temperature, y::Temperature) = +(promote(x,y)...)
-(x::Temperature, y::Temperature) = -(promote(x,y)...)

- (generic function with 216 methods)

Demo :

In [15]:
Celsius(1) + Celsius(3)
Fahrenheit(3) + Celsius(4) + Kelvin(4)
Fahrenheit(3) + Kelvin(3)

Kelvin(260.0388888888889)